In [1]:
pip install plotly==4.12.0

     |████████████████████████████████| 13.1MB 322kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [2]:
pip install plotly.express==0.4.1

In [3]:
pip install html5lib

In [4]:
pip install boto3

     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 71kB 5.3MB/s 
     |████████████████████████████████| 6.7MB 7.9MB/s 
     |████████████████████████████████| 133kB 46.1MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [5]:
pip install bs4

# How to Create Auto-Updating Data Visualizations in Python with AWS

This notebook collects the usa eleactona data from, 'https://eu.usatoday.com/elections/results/race/2020-11-03-presidential/, creates a a data frame with the respective results, and uplode the data frame to AWS following https://www.freecodecamp.org/news/how-to-create-auto-updating-data-visualizations-in-python-with-matplotlib-and-aws/ tutorial from Nick McCullum

In [6]:
#import libraries 
import bs4 as bs 
import urllib.request
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import boto3
import plotly.express as px
from io import StringIO

## Data scraping 


In [7]:
# list of unknow election results 
states=['AZ', 'FL', 'GA', 'IA', 'ME', 'MI', 'NC', 'NV', 'OH', 'PA', 'TX', 'WI']

In [8]:
#create a lists   to save urls 
urls=[]
# loop through state list and append the state to the url 
for state in states:
  url = "https://eu.usatoday.com/elections/results/race/2020-11-03-presidential-"
  url += state +'-0/'
  urls.append(url)


In [9]:
# see where to cut url to obtain back states 
x = 'https://eu.usatoday.com/elections/results/race/2020-11-03-presidential-AZ-0/'

In [10]:
x[71:73]

'AZ'

In [11]:
urls

['https://eu.usatoday.com/elections/results/race/2020-11-03-presidential-AZ-0/',
 'https://eu.usatoday.com/elections/results/race/2020-11-03-presidential-FL-0/',
 'https://eu.usatoday.com/elections/results/race/2020-11-03-presidential-GA-0/',
 'https://eu.usatoday.com/elections/results/race/2020-11-03-presidential-IA-0/',
 'https://eu.usatoday.com/elections/results/race/2020-11-03-presidential-ME-0/',
 'https://eu.usatoday.com/elections/results/race/2020-11-03-presidential-MI-0/',
 'https://eu.usatoday.com/elections/results/race/2020-11-03-presidential-NC-0/',
 'https://eu.usatoday.com/elections/results/race/2020-11-03-presidential-NV-0/',
 'https://eu.usatoday.com/elections/results/race/2020-11-03-presidential-OH-0/',
 'https://eu.usatoday.com/elections/results/race/2020-11-03-presidential-PA-0/',
 'https://eu.usatoday.com/elections/results/race/2020-11-03-presidential-TX-0/',
 'https://eu.usatoday.com/elections/results/race/2020-11-03-presidential-WI-0/']

In [12]:
# create empty list to append candidate information 
table_list=[]

# loop through urls 
for url in urls:
  # return source code 
  sauce = urllib.request.urlopen(url).read()
  #return bs object
  soup = bs.BeautifulSoup(sauce,"lxml")
  # scrape table 
  table = soup.table
  # find all the rows 
  table_rows = table.find_all('tr')
  
  # loop through rows 
  for tr in table_rows:
    # find all the td 
    td = tr.find_all('td')
    # get text for each row and column
    row = [i.text for i in td]
    # append to list 
    table_list.append(row)

  

In [13]:
# see our list 
table_list[:4]

[[],
 ['Joe\nBiden (D)', '1,655,192', '49.4%'],
 ['Donald\nTrump (R) *', '1,642,379', '49.1%'],
 ['Jo\nJorgensen (LB)', '50,636', '1.5%']]

## Data Exploration 

In [14]:
# create a function to cretate candidate data frame 
# create empty list 
def candidate_df(name):
  """
  Takes list_name as in `table_list` candidate name 
  and name of candate and returns a data frame with 
  the relevant info 
  """
  list_name =[]
  # loop through ou list and retrivr only biden info 
  for item in table_list:
    if name in item:
      list_name.append(item)
  #create data frame 
  df= pd.DataFrame(list_name, columns = ['Candidate', 'Total votes', 'Percentage of votes'])
  # add state info 
  df['State'] = states
  return df


In [15]:
# create biden df 
df_biden = candidate_df('Joe\nBiden (D)')

In [16]:
#check created data frame 
df_biden

,Candidate,Total votes,Percentage of votes,State
0,Joe\nBiden (D),"1,655,192",49.4%,AZ
1,Joe\nBiden (D),"5,284,453",47.9%,FL
2,Joe\nBiden (D),"2,469,842",49.5%,GA
3,Joe\nBiden (D),"758,352",45.0%,IA
4,Joe\nBiden (D),"430,023",52.9%,ME
5,Joe\nBiden (D),"2,790,648",50.6%,MI
6,Joe\nBiden (D),"2,663,635",48.7%,NC
7,Joe\nBiden (D),"671,427",50.2%,NV
8,Joe\nBiden (D),"2,603,731",45.2%,OH
9,Joe\nBiden (D),"3,376,367",49.8%,PA


In [17]:
# create trum dataframe 
df_trump = candidate_df('Donald\nTrump (R) *')

In [18]:
#check trum df
df_trump

,Candidate,Total votes,Percentage of votes,State
0,Donald\nTrump (R) *,"1,642,379",49.1%,AZ
1,Donald\nTrump (R) *,"5,658,847",51.2%,FL
2,Donald\nTrump (R) *,"2,457,191",49.3%,GA
3,Donald\nTrump (R) *,"897,140",53.2%,IA
4,Donald\nTrump (R) *,"359,502",44.2%,ME
5,Donald\nTrump (R) *,"2,644,525",47.9%,MI
6,Donald\nTrump (R) *,"2,736,929",50.1%,NC
7,Donald\nTrump (R) *,"634,701",47.5%,NV
8,Donald\nTrump (R) *,"3,074,418",53.4%,OH
9,Donald\nTrump (R) *,"3,328,801",49.1%,PA


Data frame with states, party, number of electoral votes was previously created 

In [19]:
#firts itereation done manually,  upload `from your computer`
#read data frame to be updated 
# from google.colab import files 
# files.upload()

In [20]:
# make a df with our new data 
# df = pd.read_csv('politics_usa.csv')

In [21]:
# if csv file is already in S3 bucket read the file from the desited url
df = pd.read_csv('https://usa-election-bucket.s3.amazonaws.com/politics_new.csv')

In [23]:
#check uploaded correctly
df.tail()

,state,party,electoral votes
46,NV,unsure,6
47,OH,unsure,18
48,PA,unsure,20
49,TX,unsure,38
50,WI,unsure,10


In [24]:
#check new df
len(df)

51

In [25]:
#split data frame into parties where the reslut is clear and where no 

df_unsure = df.loc[df['state'].isin(states)]
# drop party column to create a new one depending on results 
df_unsure = df_unsure.drop('party', axis = 1)
df_unsure

,state,electoral votes
39,AZ,11
40,FL,29
41,GA,16
42,IA,6
43,ME,4
44,MI,16
45,NC,15
46,NV,6
47,OH,18
48,PA,20


In [26]:
#create a secon data frame with the rest of the information
df_sure = df.loc[~df['state'].isin(states)]
df_sure.tail()

,state,party,electoral votes
34,SD,republican,3
35,TN,republican,11
36,UT,republican,6
37,WV,republican,5
38,WY,republican,3


In [27]:
# make a list for comparision 
biden_votes = list(df_biden['Percentage of votes'])
trump_votes = list(df_trump['Percentage of votes'])

In [28]:
# test how to get pecntage of votes for futur comparision 
df_biden.loc[df_biden['State'] == 'AZ', ['Percentage of votes']] < df_trump.loc[df_trump['State'] == 'AZ', ['Percentage of votes']] 

,Percentage of votes
0,False


In [29]:
# loop trhoug lists 
party=[]
#loop trhough votes lists to create a list with party name  in case of candidate andvantage 
for vote in range(len(biden_votes)):
  # if biden has more pecentage of votes append democrat
  if biden_votes[vote] > trump_votes[vote]:
    party.append('democrat')
  # if trump has more percentage append republican 
  elif biden_votes[vote] < trump_votes[vote]: 
    party.append('republican')
  # if hey are equal append unsure 
  else:
    party.append('unsure')
party
  

['democrat',
 'republican',
 'democrat',
 'republican',
 'democrat',
 'democrat',
 'republican',
 'democrat',
 'republican',
 'democrat',
 'republican',
 'democrat']

In [30]:
# add new results to df_unsure 
df_unsure['party'] = party


In [31]:
df_unsure

,state,electoral votes,party
39,AZ,11,democrat
40,FL,29,republican
41,GA,16,democrat
42,IA,6,republican
43,ME,4,democrat
44,MI,16,democrat
45,NC,15,republican
46,NV,6,democrat
47,OH,18,republican
48,PA,20,democrat


In [32]:
df = pd.concat([df_sure, df_unsure])
df

,state,party,electoral votes
0,CA,democrat,55
1,CO,democrat,9
2,CT,democrat,7
3,DC,democrat,3
4,DE,democrat,3
5,HI,democrat,4
6,IL,democrat,20
7,MA,democrat,11
8,MD,democrat,10
9,MN,democrat,10


In [33]:
#save dataframe 
df.to_csv('politics1.csv', index = False )
df.tail()

,state,party,electoral votes
46,NV,democrat,6
47,OH,republican,18
48,PA,democrat,20
49,TX,republican,38
50,WI,democrat,10


In [34]:
# divide df btw republican and democrats 
df_republican = df.loc[df['party'] == 'republican']
df_democrat = df.loc[df['party'] == 'democrat']

In [35]:
df_democrat

,state,party,electoral votes
0,CA,democrat,55
1,CO,democrat,9
2,CT,democrat,7
3,DC,democrat,3
4,DE,democrat,3
5,HI,democrat,4
6,IL,democrat,20
7,MA,democrat,11
8,MD,democrat,10
9,MN,democrat,10


In [36]:
df['electoral votes'].sum()

538

## Visualize Data

In [37]:
#plot map 
fig_map = px.choropleth(
                          df, locations="state", hover_name='electoral votes',
                          locationmode="USA-states", color="party",
                          scope="usa", title = 'Election Map', color_discrete_map ={'democrat': 'blue',
                                                          'republican': 'red',
                                                          'unsure': 'grey'})


fig_map.show()


In [38]:
df_test=df.head(3)
t=df_test['electoral votes'].sum()

In [39]:
fig_bar = px.histogram(df, x='party', y='electoral votes', histfunc='sum',color='party',
                           range_y=[0,350], color_discrete_map={'democrat': 'blue',
                                                                'republican': 'red'},
                           title = 'Amount of votes to win ')
fig_bar.update_layout(showlegend=False, shapes=[
        dict(type='line', yref='paper',y0=0.77,y1=0.77, xref='x',x0=-0.5,x1=1.5)
    ])
    # add annotation text above line
fig_bar.add_annotation(x=0.5, y=280, showarrow=False, text="270 votes to win")
                       
fig_bar.show()

## Upload file to AWS S3 bucket

Upload_file method of boto3 takes several arguments:

* File name -  the name of the file on our local machine.
* S3 bucket name - the name of the S3 bucket that we want to upload to.
* File name - the name that we want the file to have after it is uploaded to the AWS S3 bucket. In this case, it is the same as the first argument, but it doesn't have to be.
* ExtraArgs={'ACL':'public-read'} means that the file should be readable by the public once it is pushed to the AWS S3 bucket.

In [42]:
s3 = boto3.resource(
    service_name ='s3',
    aws_access_key_id='Your AWS key id',
    aws_secret_access_key='your AWS secret access key'
)
s3.meta.client.upload_file('politics1.csv', 'your AWS bucket name', 'politics_new.csv', ExtraArgs={'ACL':'public-read'})

## Conect code to EC2

Atlternative way from turorial : https://colab.research.google.com/github/d2l-ai/d2l-en-colab/blob/master/chapter_appendix-tools-for-deep-learning/aws.ipynb

https://dataschool.com/data-modeling-101/running-jupyter-notebook-on-an-ec2-server/

As operating system is windows the best option other than using a virtual linux enviroment(which is by far the best option) is following this tutorial: https://www.youtube.com/watch?v=qUHQuZjTOFA&ab_channel=Dr.VytautasBielinskas


how to run crontab comand : follow tutorial https://www.youtube.com/watch?v=llUw3RtD-Yw&ab_channel=IDGTECHtalk
Open terminal...
* To create new task...  crontab -e
* Press "i" to enable you to Insert a new task (look for word "INSERT" at bottom after pressing "i")
* Create task. For example: $ * * * * * echo 'hello' >> /tmp/test.txt
* Then press "esc", type ":wq" (you'll see ":wq" appear at bottom left corner), press "enter"
*  crontab -l (to confirm that new task is there)
*  cd /tmp (to change your directory in terminal to tmp)
*  ls (this will list files in tmp, you may need to wait up to a minute until you can see test.txt)
*  cat test.txt (this tells terminal to print whatever is in the file)
>> hello
>> hello
* rejoice!


Following this proccess the Ec2 instance was able to run the code every hour updating the data. NOTE: the code in EC2 doesnt run in a jupyter notebook as the configuration proccess is totally differnet. 